# Fine-Tuned DistilBERT LLM for Clinical Prediction

## 📚 Package Imports

In [25]:
# Data handling
import pandas as pd
import numpy as np
from datasets import Dataset

# LLM fine-tuning
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, recall_score, precision_score

## 📂 Loading Data

In [26]:
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

train_df.head()

,subject_id,hadm_id,hospital_expire_flag,gender,age,LoS,ER_LoS,admission_no,days_since_discharge,days_until_death,...,ccs_emb_21,ccs_emb_22,ccs_emb_23,ccs_emb_24,ccs_emb_25,ccs_emb_26,ccs_emb_27,ccs_emb_28,ccs_emb_29,ccs_emb_30
0,10000690,26504700,0,1,86,4.538889,0.205556,1,NaN,575.884722,...,-0.027652,-0.013480,0.015762,-0.035427,-0.006714,0.001981,0.031827,0.017102,-0.015162,0.023146
1,10000690,23280645,0,1,86,7.751389,0.210417,2,71.170833,500.175000,...,-0.003314,-0.001401,-0.034529,0.016691,-0.017230,0.000959,0.014027,0.013297,-0.007667,-0.001504
2,10000690,25860671,0,1,86,9.821528,0.330556,3,39.175000,453.248611,...,-0.003084,-0.001364,-0.001827,-0.000033,-0.009749,-0.007124,0.024730,0.027403,0.007163,0.014003
3,10000690,26146595,0,1,88,1.677778,0.403472,4,442.413194,1.013889,...,0.006004,-0.001361,-0.028550,-0.003648,-0.033128,-0.001682,0.002926,0.007493,-0.011065,0.013586
4,10001919,29897682,0,0,59,1.574306,NaN,1,NaN,244.000000,...,-0.008147,-0.008002,0.027353,0.016642,-0.028913,0.006824,0.018412,0.002947,-0.018814,-0.017418


## Data Preparation for `hospital_expiry_flag`

In [27]:
# Removing Label Leaking Features
train_df.drop(columns="days_until_death", inplace=True, errors='ignore')
test_df.drop(columns="days_until_death", inplace=True, errors='ignore')

# Partitions for features and label
label_col = "hospital_expire_flag"
feature_cols = [c for c in train_df.columns if c != label_col]

def row_to_text(row):
    return " | ".join([f"{col}={row[col]}" for col in feature_cols])

train_df["text"] = train_df.apply(row_to_text, axis=1)
test_df["text"] = test_df.apply(row_to_text, axis=1)

train_df[["text", label_col]].head()

,text,hospital_expire_flag
0,subject_id=10000690.0 | hadm_id=26504700.0 | g...,0
1,subject_id=10000690.0 | hadm_id=23280645.0 | g...,0
2,subject_id=10000690.0 | hadm_id=25860671.0 | g...,0
3,subject_id=10000690.0 | hadm_id=26146595.0 | g...,0
4,subject_id=10001919.0 | hadm_id=29897682.0 | g...,0


## Conversion to HuggingFace Data

In [28]:
train_ds = Dataset.from_pandas(train_df[["text", label_col]])
test_ds = Dataset.from_pandas(test_df[["text", label_col]])

train_ds = train_ds.rename_column(label_col, "label")
test_ds = test_ds.rename_column(label_col, "label")

## 🔤 Tokenization

In [29]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=256)

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map: 100%|██████████| 2430/2430 [00:01<00:00, 1777.69 examples/s]


## 🧠 Load Model

In [30]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 📏 Metrics

In [31]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    probs = torch.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "roc_auc": roc_auc_score(labels, probs),
    }

## ⚙️ Training Arguments

In [32]:
training_args = TrainingArguments(
    output_dir="/jupyter/work/module-3/group-project/llm",
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="/jupyter/work/module-3/group-project/llm/logs",
    load_best_model_at_end=True,
    metric_for_best_model="roc_auc"
)

## 🚀 Train

In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


C:\Users\migue\AppData\Local\Temp\ipykernel_31716\99649773.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\migue\Desktop\TARNIB\TARNIB\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## 📊 Evaluate

In [ ]:
metrics = trainer.evaluate()
metrics

c:\Users\migue\Desktop\TARNIB\TARNIB\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.10811549425125122,
 'eval_accuracy': 0.977366255144033,
 'eval_f1': 0.0,
 'eval_roc_auc': 0.6188708133971292,
 'eval_runtime': 412.1492,
 'eval_samples_per_second': 5.896,
 'eval_steps_per_second': 0.369,
 'epoch': 3.0}

## 🧮 Confusion Matrix

In [ ]:
preds = trainer.predict(test_ds)

logits = preds.predictions
labels = preds.label_ids

y_pred = np.argmax(logits, axis=1)

print(confusion_matrix(labels, y_pred))

c:\Users\migue\Desktop\TARNIB\TARNIB\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[[2375    0]
 [  55    0]]


## 📈 Tweaking Threshold for Maximum 

In [ ]:
# Search for max F1 score within bounds 0.02195 to 0.02197
thresholds = np.arange(0.02195, 0.021971, 0.000001)
f1_scores_search = []
best_f1 = -1
best_threshold_f1 = None

for thresh in thresholds:
    y_pred_thresh = (probs >= thresh).astype(int)
    f1 = f1_score(labels, y_pred_thresh)
    f1_scores_search.append(f1)
    
    if f1 > best_f1:
        best_f1 = f1
        best_threshold_f1 = thresh

# Apply best threshold
y_pred_best_f1 = (probs >= best_threshold_f1).astype(int)

print(f"Best threshold for max F1: {best_threshold_f1:.5f}")
print(f"Max F1 Score: {best_f1:.4f}\n")

# Confusion matrix
cm = confusion_matrix(labels, y_pred_best_f1)
print("Confusion Matrix:")
print(cm)
print()

# Calculate sensitivity and specificity
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)  # Recall / True Positive Rate
specificity = tn / (tn + fp)  # True Negative Rate

print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

Best threshold for max F1: 0.02196
Max F1 Score: 0.1018

Confusion Matrix:
[[2169  206]
 [  41   14]]

Sensitivity (Recall): 0.2545
Specificity: 0.9133
